# pyBela Tutorial 3: Logger
This notebook is a tutorial for the Logger class in the pyBela python library. As opposed to the Streamer, the Logger stores variable values directly in binary files in the Bela board. This is more reliable than streaming data with the Streamer with the saving mode enabled, which depends on the websocket connection. The Logger will store the data in Bela even if the websocket connection is lost, and you can retrieve the data later. 

As with the previous tutorials, you will need to run the `potentiometers` project in Bela. If you haven't done it yet, copy the project onto Bela: 

In [ ]:
!scp -r ../bela-code/potentiometers root@bela.local:Bela/projects

and connect the two potentiometers to the Bela analog inputs 0 and 1 (a schematic can be found in the `1_Streamer.ipynb` tutorial). Once the circuit is set up, run the `potentiometers` program in Bela, and then run the cells below to connect to the Logger to Bela and start logging data:

In [1]:
import asyncio
import os
import pandas as pd
from pyBela import Logger

In [2]:
logger = Logger()
logger.connect()

'Connection successful'

### Logging files with automatic transfer

Similarly to the Streamer and the Monitor, we can start and stop a logging session with `start_logging()` and `stop_logging`, and use asyncio to time the logging session:

In [3]:
file_paths = logger.start_logging(
    variables=["pot1", "pot2"])
await asyncio.sleep(0.5)
logger.stop_logging()

Transferring /root/Bela/projects/potentiometers/pot2.bin-->./pot2.bin... Done.
Transferring /root/Bela/projects/potentiometers/pot1.bin-->./pot1.bin... Done.


### Loading the data from the binary file
The Logger automatically transfers the files from Bela to the computer whilst the logging session is happening. This avoids long waiting times at the end of the session. 

`start_logging()` returns `file_paths`, a dictionary containing the paths in Bela of the files generated and the local paths (in this computer) to which the files are copied. You can use these paths to automate the processing of the data:

In [4]:
data = {}
for var in ["pot1", "pot2"]:
    data[var] = logger.read_binary_file(
        file_path=file_paths["local_paths"][var], timestamp_mode=logger.get_prop_of_var(var, "timestamp_mode"))

You might notice that the last buffer of each variable have 0.0 values at the end. This is because the buffers send by Bela to the Logger have a fixed size, and the last buffer might not be completely filled, so the remaining values are filled with 0.0. 

In [5]:
data["pot1"]["buffers"][-1]

{'ref_timestamp': 14495352,
 'data': [0.4510498046875,
  0.45111083984375,
  0.4509735107421875,
  0.4510650634765625,
  0.4510498046875,
  0.4510955810546875,
  0.451080322265625,
  0.4510345458984375,
  0.4510040283203125,
  0.451171875,
  0.4511260986328125,
  0.4510345458984375,
  0.4510650634765625,
  0.4510040283203125,
  0.4510955810546875,
  0.451141357421875,
  0.4510345458984375,
  0.4509124755859375,
  0.4510345458984375,
  0.45111083984375,
  0.4510345458984375,
  0.4510650634765625,
  0.451080322265625,
  0.4510498046875,
  0.4511871337890625,
  0.4510498046875,
  0.4510498046875,
  0.4511260986328125,
  0.4510650634765625,
  0.4510650634765625,
  0.451019287109375,
  0.4510955810546875,
  0.451141357421875,
  0.4510345458984375,
  0.45098876953125,
  0.4510955810546875,
  0.4510650634765625,
  0.45111083984375,
  0.4510498046875,
  0.4510498046875,
  0.4510498046875,
  0.4510345458984375,
  0.4510650634765625,
  0.4510345458984375,
  0.45098876953125,
  0.4510498046875,
 

You can also flatten the buffers using:

In [6]:
flatten_data = {}
for var in ["pot1", "pot2"]:
    flatten_data[var] = {"timestamps": [], "data": []}
    for _buffer in data[var]["buffers"]:
        flatten_data[var]["timestamps"].extend([_buffer["ref_timestamp"] + i for i in range(len(_buffer["data"]))])
        flatten_data[var]["data"].extend(_buffer["data"])
    
df = pd.DataFrame(flatten_data["pot1"])
df.head()

,timestamps,data
0,14482040,0.451035
1,14482041,0.450989
2,14482042,0.451065
3,14482043,0.451080
4,14482044,0.451080


Whilst the Logger by default transfers the binary files to Bela, the files are not removed from Bela. You can remove the files from Bela with `delete_file_from_bela()`:

In [7]:
for file in file_paths["remote_paths"].values():
    logger.delete_file_from_bela(file, verbose=True)

File '/root/Bela/projects/potentiometers/pot1.bin' has been removed from Bela.
File '/root/Bela/projects/potentiometers/pot2.bin' has been removed from Bela.


### Logging files without automatic transfer

Alternatively, you can set `transfer=False` in `start_logging()` and transfer the files manually with `copy_file_from_bela()`:

In [8]:
file_paths =  logger.start_logging(
    variables= ["pot1", "pot2"], transfer=False)
await asyncio.sleep(0.5)
logger.stop_logging()

file_paths

{'local_paths': {},
 'remote_paths': {'pot1': '/root/Bela/projects/potentiometers/pot1.bin',
  'pot2': '/root/Bela/projects/potentiometers/pot2.bin'}}

Note that the dict in `local_paths` is empty: since we disabled the automatic transfer, no local paths have been assigned. We can now copy the files using `copy_file_from_bela()`:

In [9]:
for var in ["pot1", "pot2"]:
    logger.copy_file_from_bela(remote_path=file_paths["remote_paths"][var], local_path=os.path.basename(file_paths["remote_paths"][var]))

pot1.bin already exists. Renaming file to pot1_1.bin
pot2.bin already exists. Renaming file to pot2_1.bin


You might end up with a few `.bin` files in Bela. You can either remove them one by one with `delete_file_from_bela()` as explained above, or remove all `.bin` files in the Bela project with `delete_all_bin_files_in_project()``

In [10]:
logger.delete_all_bin_files_in_project()